In [1]:
import abce
import abcFinance
from account import AccountSide

/Users/taghawi/Dropbox/workspace/abce/message_plugin/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [2]:
class Bank(abcFinance.Agent):
    def init(self, reserves=0, loans=0, deposits=0):
        self.make_asset_accounts(['reserves', 'loans'])
        self.make_liability_accounts(['deposits'])
        self.make_flow_accounts(['interest income'])
        #self.make_residual_account('equity')
        self.book(debit=[('reserves', reserves), ('loans', loans)], 
                  credit=[('deposits', deposits), ('equity', reserves + loans - deposits)], 
                  text='Capital endowment')
        self.book_end_of_period()
        
class Household(abcFinance.Agent):
    def init(self, money=0, loans=0, participations=0):
        self.make_asset_accounts(['money','participations'])
        self.make_liability_accounts(['loans'])
        self.make_flow_accounts(['dividend income','interest expenses'])
        #self.make_residual_account('equity')
        self.book(debit=[('money', money),('participations', participations)], 
                  credit=[('loans', loans), ('equity', money + participations - loans)], 
                  text='Initial endowment')
        self.book_end_of_period()

## Initialize the system with outside money endowments

In [3]:
sim = abce.Simulation()
bank = sim.build_agents(Bank, 'bank', number=1, reserves=100,deposits=100)
household = sim.build_agents(Household, 'household', number=1, money=100, loans=0)
bank_owner = sim.build_agents(Household, 'bank_owner', number=1, participations=100)
print('Bank')
bank.print_balance_sheet()
print('Household')
household.print_balance_sheet()
print('Bank owner')
bank_owner.print_balance_sheet()

Bank
Asset accounts:
   reserves : 100
Liability accounts:
   deposits : 100
Equity:  0
Total Assets:  100
--
Household
Asset accounts:
   money : 100
Liability accounts:
Equity:  100
Total Assets:  100
--
Bank owner
Asset accounts:
   participations : 100
Liability accounts:
Equity:  100
Total Assets:  100
--


## Granting of a loan

In [4]:
bank.book(debit=[('loans', 100)], credit=[('deposits', 100)], text='Loan granting')
household.book(debit=[('money', 100)], credit=[('loans', 100)], text='Take out loan')
print('Bank')
bank.print_balance_sheet()
print('Household')
household.print_balance_sheet()

Bank
Asset accounts:
   reserves : 100
   loans : 100
Liability accounts:
   deposits : 200
Equity:  0
Total Assets:  200
--
Household
Asset accounts:
   money : 200
Liability accounts:
   loans : 100
Equity:  100
Total Assets:  200
--


## Interest payment

In [5]:
household.book(debit=[('interest expenses', 10)], credit=[('money', 10)], text='Interest payment')
bank.book(debit=[('deposits', 10)], credit=[('interest income', 10)], text='Interest payment')
print('Bank')
bank.print_balance_sheet()
bank.print_profit_and_loss()
print('Household')
household.print_balance_sheet()
household.print_profit_and_loss()

Bank
Asset accounts:
   reserves : 100
   loans : 100
Liability accounts:
   deposits : 190
Equity:  0
Total Assets:  200
--
Flow accounts:
   interest income : 10
Profit for period:  10
--
Household
Asset accounts:
   money : 190
Liability accounts:
   loans : 100
Equity:  100
Total Assets:  190
--
Flow accounts:
   interest expenses : -10
Profit for period:  -10
--


## Dividend payment

In [6]:
bank.book(debit=[('equity', 10)], credit=[('deposits', 10)], text='Dividend payment')
bank_owner.book(debit=[('money', 10)], credit=[('dividend income', 10)], text='Dividend payment')
print('Bank')
bank.print_balance_sheet()
bank.print_profit_and_loss()
print('Bank owner')
bank_owner.print_balance_sheet()
bank_owner.print_profit_and_loss()

Bank
Asset accounts:
   reserves : 100
   loans : 100
Liability accounts:
   deposits : 200
Equity:  -10
Total Assets:  200
--
Flow accounts:
   interest income : 10
Profit for period:  10
Profit distribution and capital actions
   Dividend payment : -10
--
Bank owner
Asset accounts:
   money : 10
   participations : 100
Liability accounts:
Equity:  100
Total Assets:  110
--
Flow accounts:
   dividend income : 10
Profit for period:  10
--


## Loan principal repayment

In [7]:
household.book(debit=[('loans', 100)], credit=[('money', 100)], text='Principal repayment')
bank.book(debit=[('deposits', 100)], credit=[('loans', 100)], text='Principal repayment')
print('Bank')
bank.print_balance_sheet()
print('Household')
household.print_balance_sheet()
print('Bank owner')
bank_owner.print_balance_sheet()

Bank
Asset accounts:
   reserves : 100
Liability accounts:
   deposits : 100
Equity:  -10
Total Assets:  100
--
Household
Asset accounts:
   money : 90
Liability accounts:
Equity:  100
Total Assets:  90
--
Bank owner
Asset accounts:
   money : 10
   participations : 100
Liability accounts:
Equity:  100
Total Assets:  110
--
